In [1]:
import tensorflow as tf
tf.logging.set_verbosity(tf.logging.WARN)
import pickle
import numpy as np
import os
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
import os
from tensorflow.python.client import device_lib
from collections import Counter

In [2]:
f = open('../Glove/word_embedding_glove', 'rb')
word_embedding = pickle.load(f)
f.close()
word_embedding = word_embedding[: len(word_embedding)-1]

f = open('../Glove/vocab_glove', 'rb')
vocab = pickle.load(f)
f.close()

word2id = dict((w, i) for i,w in enumerate(vocab))
id2word = dict((i, w) for i,w in enumerate(vocab))

unknown_token = "UNKNOWN_TOKEN"

In [3]:
f = open("../../dataset/sense/dict_sense-keys", 'rb')
dict_sense_keys = pickle.load(f)
f.close()

f = open("../../dataset/sense/dict_word-sense", 'rb')
dict_word_sense = pickle.load(f)
f.close()

In [4]:
# Model Description
sense_word = 'force'
model_name = 'model-3'
sense_word_dir = 'output/' + sense_word
model_dir = sense_word_dir + '/' + model_name
save_dir = os.path.join(model_dir, "save/")
log_dir = os.path.join(model_dir, "log")

if not os.path.exists(sense_word_dir):
    os.mkdir(sense_word_dir)
if not os.path.exists(model_dir):
    os.mkdir(model_dir)
if not os.path.exists(save_dir):
    os.mkdir(save_dir)
if not os.path.exists(log_dir):
    os.mkdir(log_dir)

In [5]:
f = open("../../dataset/checkwords/"+ sense_word + "_data", 'rb')
data = pickle.load(f)
f.close()

pos_tags = []
for i in range(len(data)):
    for p in data[i][2]:
        pos_tags.append(p)
        
pos_count = Counter(pos_tags)
pos_count = pos_count.most_common()
vocab_pos = [k for k,v in pos_count]

data_y = []
for i in range(len(data)):
    data_y.append(dict_sense_keys[data[i][0]][3])

sense_count = Counter(data_y)
sense_count = sense_count.most_common()
vocab_sense = [k for k,v in sense_count[:5]]
sense_count

[('1:14', 1150),
 ('1:07', 969),
 ('2:41', 543),
 ('1:04', 495),
 ('1:18', 492),
 ('2:36', 31),
 ('1:19', 20),
 ('2:38', 10),
 ('2:32', 7),
 ('2:35', 6)]

In [6]:
vocab_sense

['1:14', '1:07', '2:41', '1:04', '1:18']

In [7]:
data_x = []
data_label = []
data_pos = []
for i in range(len(data)):
    if dict_sense_keys[data[i][0]][3] in vocab_sense:
        data_x.append(data[i][1])
        data_pos.append(data[i][2])
        data_label.append(dict_sense_keys[data[i][0]][3])

print(len(data_label), len(data_y))

pos2id = dict((s, i) for i,s in enumerate(vocab_pos))
id2pos = dict((i, s) for i,s in enumerate(vocab_pos))

# vocab_sense = dict_word_sense[sense_word]

sense2id = dict((s, i) for i,s in enumerate(vocab_sense))
id2sense = dict((i, s) for i,s in enumerate(vocab_sense))

3649 3723


In [8]:
# Parameters
mode = 'train'
num_senses = len(vocab_sense)
num_pos = len(vocab_pos)
batch_size = 64
vocab_size = len(vocab)
unk_vocab_size = 1
word_emb_size = len(word_embedding[0])
max_sent_size = 200
hidden_size = 100
keep_prob = 0.5
l2_lambda = 0.001
init_lr = 0.005
decay_steps = 500
decay_rate = 0.96
clip_norm = 1
clipping = True
crf_lambda = 0.1

In [9]:
# MODEL
x = tf.placeholder('int32', [batch_size, max_sent_size], name="x")
y = tf.placeholder('int32', [batch_size], name="y")
y_pos = tf.placeholder('int32', [batch_size, None], name="y_pos")
x_mask  = tf.placeholder('bool', [batch_size, max_sent_size], name='x_mask') 
is_train = tf.placeholder('bool', [], name='is_train')
word_emb_mat = tf.placeholder('float', [None, word_emb_size], name='emb_mat')
input_keep_prob = tf.cond(is_train,lambda:keep_prob, lambda:tf.constant(1.0))
x_len = tf.reduce_sum(tf.cast(x_mask, 'int32'), 1)

with tf.name_scope("word_embedding"):
    if mode == 'train':
        unk_word_emb_mat = tf.get_variable("word_emb_mat", dtype='float', shape=[unk_vocab_size, word_emb_size], initializer=tf.contrib.layers.xavier_initializer(uniform=True, seed=0, dtype=tf.float32))
    else:
        unk_word_emb_mat = tf.get_variable("word_emb_mat", shape=[unk_vocab_size, word_emb_size], dtype='float')
        
    final_word_emb_mat = tf.concat([word_emb_mat, unk_word_emb_mat], 0)
    Wx = tf.nn.embedding_lookup(final_word_emb_mat, x)  

with tf.variable_scope("lstm1"):
    cell_fw1 = tf.contrib.rnn.BasicLSTMCell(hidden_size,state_is_tuple=True)
    cell_bw1 = tf.contrib.rnn.BasicLSTMCell(hidden_size,state_is_tuple=True)

    d_cell_fw1 = tf.contrib.rnn.DropoutWrapper(cell_fw1, input_keep_prob=input_keep_prob)
    d_cell_bw1 = tf.contrib.rnn.DropoutWrapper(cell_bw1, input_keep_prob=input_keep_prob)
    
    (fw_h1, bw_h1), _ = tf.nn.bidirectional_dynamic_rnn(d_cell_fw1, d_cell_bw1, Wx, sequence_length=x_len, dtype='float', scope='lstm1')
    h1 = tf.concat([fw_h1, bw_h1], 2)
    
with tf.variable_scope("lstm2"):
    cell_fw2 = tf.contrib.rnn.BasicLSTMCell(hidden_size,state_is_tuple=True)
    cell_bw2 = tf.contrib.rnn.BasicLSTMCell(hidden_size,state_is_tuple=True)

    d_cell_fw2 = tf.contrib.rnn.DropoutWrapper(cell_fw2, input_keep_prob=input_keep_prob)
    d_cell_bw2 = tf.contrib.rnn.DropoutWrapper(cell_bw2, input_keep_prob=input_keep_prob)
    
    (fw_h2, bw_h2), _ = tf.nn.bidirectional_dynamic_rnn(d_cell_fw2, d_cell_bw2, h1, sequence_length=x_len, dtype='float', scope='lstm2')
    h = tf.concat([fw_h2, bw_h2], 2)

def attention(input_x, input_mask, W_att):
    h_masked = tf.boolean_mask(input_x, input_mask)
    h_tanh = tf.tanh(h_masked)
    u = tf.matmul(h_tanh, W_att)
    a = tf.nn.softmax(u)
    c = tf.reduce_sum(tf.multiply(h_tanh, a), 0)  
    return c

with tf.variable_scope("attention"):
    W_att = tf.Variable(tf.truncated_normal([2*hidden_size, 1], mean=0.0, stddev=0.1, seed=0), name="W_att")
    c = tf.expand_dims(attention(h[0], x_mask[0], W_att), 0)
    for i in range(1, batch_size):
        c = tf.concat([c, tf.expand_dims(attention(h[i], x_mask[i], W_att), 0)], 0)
        
with tf.variable_scope("softmax_layer"):
    W = tf.Variable(tf.truncated_normal([2*hidden_size, num_senses], mean=0.0, stddev=0.1, seed=0), name="W")
    b = tf.Variable(tf.zeros([num_senses]), name="b")
    drop_c = tf.nn.dropout(c, input_keep_prob)
    logits = tf.matmul(drop_c, W) + b
    predictions = tf.argmax(logits, 1)
    
flat_h1 = tf.reshape(h1, [-1, 2*hidden_size])

with tf.variable_scope("crf_layer_pos"):
    W = tf.Variable(tf.truncated_normal([2*hidden_size, num_pos], mean=0.0, stddev=0.1, seed=0), name="W")
    b = tf.Variable(tf.zeros([num_pos]), name="b")
    drop_flat_h1 = tf.nn.dropout(flat_h1, input_keep_prob)
    flat_logits_pos = tf.matmul(drop_flat_h1, W) + b
    logits_pos = tf.reshape(flat_logits_pos, [batch_size, max_sent_size, num_pos])
    
    log_likelihood, trans_params = tf.contrib.crf.crf_log_likelihood(logits_pos, y_pos, x_len)
    trans_params = trans_params
    crf_loss = crf_lambda*tf.reduce_mean(-log_likelihood)
    
    
float_x_mask = tf.cast(x_mask, 'float')

loss = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=y))

global_step = tf.Variable(0, trainable=False, name="global_step")

learning_rate = tf.train.exponential_decay(init_lr, global_step, decay_steps, decay_rate, staircase=True)

tv_all = tf.trainable_variables()
tv_regu =[]
for t in tv_all:
    if t.name.find('b:')==-1:
        tv_regu.append(t)
        
# l2 Loss
l2_loss = l2_lambda * tf.reduce_sum([ tf.nn.l2_loss(v) for v in tv_regu ])

total_loss = loss + l2_loss + crf_loss

# Optimizer for loss
optimizer = tf.train.AdamOptimizer(learning_rate)
optimizer_pos = tf.train.AdamOptimizer(learning_rate).minimize(crf_loss, global_step)

# Gradients and Variables for Loss
grads_vars = optimizer.compute_gradients(total_loss)

# Clipping of Gradients
clipped_grads = grads_vars
if(clipping == True):
    clipped_grads = [(tf.clip_by_norm(grad, clip_norm), var) for grad, var in clipped_grads]

# Training Optimizer for Total Loss
train_op = optimizer.apply_gradients(clipped_grads, global_step=global_step)

# Summaries
var_summaries = []
for v in tv_all:
    var_summary = tf.summary.histogram("{}/var".format(v.name), v)
    var_summaries.append(var_summary)

var_summaries_merged = tf.summary.merge(var_summaries)

loss_summary = tf.summary.scalar("loss", loss)
loss_pos_summary = tf.summary.scalar("loss_pos", crf_loss)
total_loss_summary = tf.summary.scalar("total_loss", total_loss)
summary = tf.summary.merge_all()

In [10]:
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="3"
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)
sess.run(tf.global_variables_initializer())                          # For initializing all the variables
saver = tf.train.Saver()                                             # For Saving the model
summary_writer = tf.summary.FileWriter(log_dir, sess.graph)          # For writing Summaries

In [11]:
index = []
for i in range(len(data_x)):
    index.append(i)

In [12]:
index_train, index_val, label_train, label_val = train_test_split(index, data_label, train_size=0.8, shuffle=True, stratify=data_label, random_state=0)

data_x = np.array(data_x)
data_pos = np.array(data_pos)

x_train = data_x[index_train]
pos_train = data_pos[index_train]
x_val = data_x[index_val]
pos_val = data_pos[index_val]

/users/btech/aviraj/cs771/lib/python3.5/site-packages/sklearn/model_selection/_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [13]:
def data_prepare(x, y, p):
    num_examples = len(x)

    xx = np.zeros([num_examples, max_sent_size], dtype=int)
    xx_mask = np.zeros([num_examples, max_sent_size], dtype=bool)
    yy = np.zeros([num_examples], dtype=int)
    pp = np.zeros([num_examples, max_sent_size], dtype=int)

    for j in range(num_examples):
        for i in range(max_sent_size):
            if(i>=len(x[j])):
                break
            w = x[j][i]
            xx[j][i] = word2id[w] if w in word2id else word2id['UNKNOWN_TOKEN']
            xx_mask[j][i] = True
            pp[j][i] = pos2id[p[j][i]]
        yy[j] = sense2id[y[j]]
        
    return xx, xx_mask, yy, pp

def model(xx, yy, mask, pp, train_cond=True, pretrain=False):
    num_batches = int(len(xx)/batch_size)
    losses = 0
    preds = []
    preds_pos = []
    true_pos = []
    for j in range(num_batches): 
        
        s = j * batch_size
        e = (j+1) * batch_size
        
        feed_dict = {x:xx[s:e], y:yy[s:e], y_pos:pp[s:e], x_mask:mask[s:e], is_train:train_cond, input_keep_prob:keep_prob, word_emb_mat:word_embedding}
        
        if(train_cond==True):
            if(pretrain==False):
                _, _loss, step, _summary = sess.run([train_op, total_loss, global_step, summary], feed_dict)
            else:
                 _, _loss, step, _summary = sess.run([optimizer_pos, crf_loss, global_step, summary], feed_dict)
            
            summary_writer.add_summary(_summary, step)
#             print("Steps:{}".format(step), ", Loss: {}".format(_loss))

        else:
            _loss, pred, crf_logits, trans_params_ = sess.run([total_loss, predictions, logits_pos, trans_params], feed_dict)
            preds.append(pred)
            for k in range(batch_size):
                logit_ = crf_logits[k][:sum(mask[s+k])] # keep only the valid steps
                viterbi_seq, viterbi_score = tf.contrib.crf.viterbi_decode(logit_, trans_params_)
                preds_pos += [viterbi_seq]    
            
            for k in range(batch_size):
                true_pos.append(pp[s+k][:sum(mask[s+k])])

        losses +=_loss

    if(train_cond==False):
        y_pred = []
        pos_true = []
        pos_pred = []
        for i in range(num_batches):
            for pred in preds[i]:
                y_pred.append(pred)
                
        for i in range(len(preds_pos)):
            for j in range(len(preds_pos[i])):
                pos_pred.append(preds_pos[i][j])
                pos_true.append(true_pos[i][j])

        return losses/num_batches, y_pred, pos_pred, pos_true
    
    return losses/num_batches, step

def eval_score(yy, pred, pos_true, pos_pred):
    num_batches = int(len(yy)/batch_size)
    f1 = f1_score(yy[:batch_size*num_batches], pred, average='macro')
    f1_pos = f1_score(pos_true, pos_pred, average='macro')
    accu = accuracy_score(yy[:batch_size*num_batches], pred)
    accu_pos = accuracy_score(pos_true, pos_pred)
    return f1*100, accu*100, f1_pos*100, accu_pos*100

In [14]:
x_id_train, mask_train, y_train, pos_id_train = data_prepare(x_train, label_train, pos_train)
x_id_val, mask_val, y_val, pos_id_val = data_prepare(x_val, label_val,pos_val)

In [15]:
num_epochs = 120
pre_train_period = 5
log_period = 5

for i in range(num_epochs):
    random = np.random.choice(len(y_train), size=(len(y_train)), replace=False)
    x_id_train = x_id_train[random]
    y_train = y_train[random]
    mask_train = mask_train[random]    
    pos_id_train = pos_id_train[random]
    
    if(i<pre_train_period):
        losses, step = model(x_id_train, y_train, mask_train, pos_id_train, pretrain=True)
    else:
        losses, step = model(x_id_train, y_train, mask_train, pos_id_train)
    print("Epoch:", i+1,"Step:", step, "loss:",losses)
    
    if((i+1)%log_period==0):
        saver.save(sess, save_path=save_dir)                         
        print("Model Saved")
        train_loss, train_pred, train_pred_pos, train_true_pos = model(x_id_train, y_train, mask_train, pos_id_train, train_cond=False)        
        f1_, accu_, f1_pos_, accu_pos_ = eval_score(y_train, train_pred, train_pred_pos, train_true_pos)
        print("Train: F1 : ",  f1_, "Accu: ", accu_, "POS F1 : ",  f1_pos_, "POS Accu: ", accu_pos_, "Loss: ", train_loss)
        
        val_loss, val_pred, val_pred_pos, val_true_pos = model(x_id_val, y_val, mask_val, pos_id_val, train_cond=False)        
        f1_, accu_, f1_pos_, accu_pos_ = eval_score(y_val, val_pred, val_pred_pos, val_true_pos)
        print("Val: F1 : ",  f1_, "Accu: ", accu_, "POS F1 : ",  f1_pos_, "POS Accu: ", accu_pos_, "Loss: ", val_loss)
        
# test_loss, test_pred, test_pred_pos, test_true_pos = model(x_id_test, y_test, mask_test, pos_id_test, train_cond=False)        
# f1_, accu_, f1_pos_, accu_pos_ = etest_score(y_test, test_pred, test_pred_pos, test_true_pos)
# print("test: F1 Score: ",  f1_, "Accuracy: ", accu_, "POS F1 Score: ",  f1_pos_, "POS Accuracy: ", accu_pos_, "Loss: ", test_loss)

Epoch: 1 Step: 45 loss: 4.10653958321
Epoch: 2 Step: 90 loss: 1.77671341631
Epoch: 3 Step: 135 loss: 1.4458066278
Epoch: 4 Step: 180 loss: 1.2627938615
Epoch: 5 Step: 225 loss: 1.14185533921
Model Saved
Train: F1 :  18.548358743 Accu:  19.4791666667 POS F1 :  82.6285666785 POS Accu:  90.6845201725 Loss:  10.4480457518


/users/btech/aviraj/cs771/lib/python3.5/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


Val: F1 :  20.3134941555 Accu:  21.7329545455 POS F1 :  81.363464465 POS Accu:  90.5016722408 Loss:  9.74386934801
Epoch: 6 Step: 270 loss: 9.00812518862
Epoch: 7 Step: 315 loss: 3.43508560922
Epoch: 8 Step: 360 loss: 2.73454948531
Epoch: 9 Step: 405 loss: 2.49110958841
Epoch: 10 Step: 450 loss: 2.35205959214
Model Saved
Train: F1 :  87.5841336342 Accu:  88.2638888889 POS F1 :  82.0149410904 POS Accu:  90.2790389048 Loss:  2.28725734817
Val: F1 :  86.3010344505 Accu:  87.2159090909 POS F1 :  81.0182595685 POS Accu:  90.1918676289 Loss:  2.39771912315
Epoch: 11 Step: 495 loss: 2.28528653251
Epoch: 12 Step: 540 loss: 2.1892815166
Epoch: 13 Step: 585 loss: 2.18985156483
Epoch: 14 Step: 630 loss: 2.11746168402
Epoch: 15 Step: 675 loss: 2.14564627806
Model Saved
Train: F1 :  89.9809446696 Accu:  90.2777777778 POS F1 :  82.6556298996 POS Accu:  90.6542627 Loss:  2.10512066152
Val: F1 :  88.4084858759 Accu:  89.0625 POS F1 :  81.7637968894 POS Accu:  90.5509593381 Loss:  2.22056700967
Epoch: 

Epoch: 97 Step: 4365 loss: 1.78225853708
Epoch: 98 Step: 4410 loss: 1.79135333432
Epoch: 99 Step: 4455 loss: 1.78929918077
Epoch: 100 Step: 4500 loss: 1.79509212441
Model Saved
Train: F1 :  94.6456368305 Accu:  95.0347222222 POS F1 :  84.061362028 POS Accu:  91.9969971281 Loss:  1.76556610001
Val: F1 :  89.4157593149 Accu:  90.4829545455 POS F1 :  83.0248670492 POS Accu:  91.7514522091 Loss:  2.03228885477
Epoch: 101 Step: 4545 loss: 1.78429352178
Epoch: 102 Step: 4590 loss: 1.80655300087
Epoch: 103 Step: 4635 loss: 1.79661542575
Epoch: 104 Step: 4680 loss: 1.79470628632
Epoch: 105 Step: 4725 loss: 1.78336937957
Model Saved
Train: F1 :  92.412452406 Accu:  92.6388888889 POS F1 :  84.389336572 POS Accu:  91.8075127612 Loss:  1.82675262557
Val: F1 :  87.8583260466 Accu:  88.2102272727 POS F1 :  83.0046111748 POS Accu:  91.5965499032 Loss:  2.08866609227
Epoch: 106 Step: 4770 loss: 1.80162519614
Epoch: 107 Step: 4815 loss: 1.79087861114
Epoch: 108 Step: 4860 loss: 1.77984184954
Epoch: 109

In [16]:
num_epochs = 10
pre_train_period = 0
log_period = 5

for i in range(num_epochs):
    random = np.random.choice(len(y_train), size=(len(y_train)), replace=False)
    x_id_train = x_id_train[random]
    y_train = y_train[random]
    mask_train = mask_train[random]    
    pos_id_train = pos_id_train[random]
    
    if(i<pre_train_period):
        losses, step = model(x_id_train, y_train, mask_train, pos_id_train, pretrain=True)
    else:
        losses, step = model(x_id_train, y_train, mask_train, pos_id_train)
    print("Epoch:", i+1,"Step:", step, "loss:",losses)
    
    if((i+1)%log_period==0):
        saver.save(sess, save_path=save_dir)                         
        print("Model Saved")
        train_loss, train_pred, train_pred_pos, train_true_pos = model(x_id_train, y_train, mask_train, pos_id_train, train_cond=False)        
        f1_, accu_, f1_pos_, accu_pos_ = eval_score(y_train, train_pred, train_pred_pos, train_true_pos)
        print("Train: F1 : ",  f1_, "Accu: ", accu_, "POS F1 : ",  f1_pos_, "POS Accu: ", accu_pos_, "Loss: ", train_loss)
        
        val_loss, val_pred, val_pred_pos, val_true_pos = model(x_id_val, y_val, mask_val, pos_id_val, train_cond=False)        
        f1_, accu_, f1_pos_, accu_pos_ = eval_score(y_val, val_pred, val_pred_pos, val_true_pos)
        print("Val: F1 : ",  f1_, "Accu: ", accu_, "POS F1 : ",  f1_pos_, "POS Accu: ", accu_pos_, "Loss: ", val_loss)
        
# test_loss, test_pred, test_pred_pos, test_true_pos = model(x_id_test, y_test, mask_test, pos_id_test, train_cond=False)        
# f1_, accu_, f1_pos_, accu_pos_ = etest_score(y_test, test_pred, test_pred_pos, test_true_pos)
# print("test: F1 Score: ",  f1_, "Accuracy: ", accu_, "POS F1 Score: ",  f1_pos_, "POS Accuracy: ", accu_pos_, "Loss: ", test_loss)

Epoch: 1 Step: 5445 loss: 1.77085706128
Epoch: 2 Step: 5490 loss: 1.7812341743
Epoch: 3 Step: 5535 loss: 1.75988491111
Epoch: 4 Step: 5580 loss: 1.7516244438
Epoch: 5 Step: 5625 loss: 1.73890060584
Model Saved
Train: F1 :  95.0947994155 Accu:  95.3472222222 POS F1 :  84.1777200682 POS Accu:  92.1484286387 Loss:  1.69059795539


/users/btech/aviraj/cs771/lib/python3.5/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


Val: F1 :  90.1856811741 Accu:  91.0511363636 POS F1 :  82.9216057002 POS Accu:  91.5930293962 Loss:  1.99522547288
Epoch: 6 Step: 5670 loss: 1.73954251342
Epoch: 7 Step: 5715 loss: 1.76034436756
Epoch: 8 Step: 5760 loss: 1.74098689026
Epoch: 9 Step: 5805 loss: 1.76669082112
Epoch: 10 Step: 5850 loss: 1.75392257373
Model Saved
Train: F1 :  95.3937543224 Accu:  95.625 POS F1 :  84.4416991939 POS Accu:  91.8614401006 Loss:  1.71533151733
Val: F1 :  90.3444766003 Accu:  91.1931818182 POS F1 :  84.14544704 POS Accu:  91.9450800915 Loss:  1.94871345433


In [15]:
saver.restore(sess, save_dir)